In [1]:
from surprise.model_selection import GridSearchCV, train_test_split
from surprise import SVDpp, Dataset, Reader, accuracy
import pickle
import pandas as pd
import gzip
import tracemalloc
import random

In [5]:
with open('../Modelo/modelo_entrenado.pkl', 'rb') as file:
    model = pickle.load(file)

In [8]:
entrenar = pd.read_csv(r'../data_set_limpio/modelo.csv.gz')

In [45]:
'''

    En este bloque realizo un merge sobre los datasets y selecciono solo las columnas que considero necesarias

'''

prueba1 = pd.merge(opinion[['item_id','user','recommend','opinion','helpful']], df[['id','app_name']], left_on='item_id', right_on='id')

prueba1['recommend'] = prueba1['recommend'].replace({True: 1, False: 0})

prueba1.loc[prueba1['helpful'] == 'No ratings yet', 'helpful'] = 0

prueba1.loc[prueba1['helpful']  != 0 , 'helpful'] = 1

In [393]:
'''

    Itero sobre cada linea para crear una nueva columna llamada ratin, esa es la que voy a utilizar para entrenar el modelo, en conjunto con app_name y user.

'''

for index, row in prueba1.iterrows():
    if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 1) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 0
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    else:
        prueba1.at[index, 'rating'] = 2
        

C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.

In [413]:
'''

    Vuelvo a filtrar el dataset, para obtener solo las columnas que necesarias para entrenar el modelo

'''

entrenar = prueba1[['user','app_name','rating']]

In [361]:
'''

    Utilizo la herramienta GridSearchCV, para buscar los mejores hiperparametros para mi modelo
    Elegi SSVDpp por que es útil cuando se trata de sistemas de recomendación basados en datos implícitos

'''

reader = Reader(rating_scale=(0, 4)) 

data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 25],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'random_state': [0, 42]
}

model = SVDpp()

grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

grid_search.fit(data)

best_params = grid_search.best_params['rmse']

print("Mejores parámetros encontrados:", best_params)

#Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.04, 'random_state': 42}

Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.04, 'random_state': 0}


In [438]:
'''

    Con los datos ya preparados y los mejores hiperparametros, solo me queda entrenar el modelo.

'''

reader = Reader(rating_scale=(0, 4))  # el rating_scale hace referencia al puntaje de mi columna rating

data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

model = SVDpp(n_factors = 50, n_epochs = 25, lr_all= 0.005, reg_all= 0.04)

model.fit(trainset)

predictions = model.test(testset)

from surprise import accuracy

rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 1.0196
RMSE: 1.0196406487549206


In [42]:
def recomendaciones_usuario(usuario: str):
    '''
    
        Esta funcion pone en marcha el modelo ya entrenado de machine learning, solo se le pasa un usuario en los requisitos
        La funcion va a retornar 5 juegos recomendados en base a los gustos del usuario, basado en su rating recomendaciones
        En caso de que el usuario ingresado no haya ralizado una compra y no tenga items en su biblioteca, se le presentaran
        5 juegos al asar, que cambiaran luego cuando se determine el gusto del usuario al comprar y realizar valoraciones de juegos.
    
    '''
    
    if usuario not in entrenar['user'].unique():
        juegos_aleatorios = list(entrenar['app_name'].sample(5))
        mensaje = f'El usuario {usuario} no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.'
        return {mensaje: juegos_aleatorios}
    else:
        diccionario = {}
        
        juegos_valorados = entrenar[entrenar['user'] == usuario]['app_name'].unique()
        todos_los_juegos = entrenar['app_name'].unique()

        juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

        predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
        
        recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 
        cont = 1
        for recomendacion in recomendaciones:
            diccionario[f'Top {cont}'] = recomendacion.iid
            cont += 1
            
        return {f'Usuario: {usuario}': diccionario}



In [48]:
'''
    Realizo un pequenio testeo
'''

testeo1 = recomendaciones_usuario('Freejam')
testeo1

{'El usuario Freejam no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria.': ['Counter-Strike: Global Offensive',
  'Dying Light',
  'Counter-Strike: Global Offensive',
  'Middle-earth™: Shadow of Mordor™',
  'Castle Crashers®']}

In [33]:
usuarios = entrenar['user'].unique() # --- > Testeo diferentes usuarios para comprobar su comportamiento

diccionario = {}

for x in usuarios[:5]:
    
    diccionario[x] = recomendaciones_usuario(x)

diccionario

{'76561197970982479': {'Usuario: 76561197970982479': {'Top 1': 'Magicite',
   'Top 2': 'Bit Blaster XL',
   'Top 3': 'The Elder Scrolls® Online: Tamriel Unlimited™',
   'Top 4': 'Tales from the Borderlands',
   'Top 5': "Sid Meier's Civilization V: Brave New World"}},
 'death-hunter': {'Usuario: death-hunter': {'Top 1': 'LEGO® Star Wars™ - The Complete Saga',
   'Top 2': 'Batman - The Telltale Series',
   'Top 3': 'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™',
   'Top 4': 'Resident Evil 6 / Biohazard 6',
   'Top 5': 'Bit Blaster XL'}},
 'DJKamBer': {'Usuario: DJKamBer': {'Top 1': 'Batman - The Telltale Series',
   'Top 2': 'Bit Blaster XL',
   'Top 3': 'Dragon Age: Origins',
   'Top 4': 'Resident Evil 6 / Biohazard 6',
   'Top 5': 'Life is Strange - Episode 1'}},
 'diego9031': {'Usuario: diego9031': {'Top 1': 'Tales from the Borderlands',
   'Top 2': 'Bit Blaster XL',
   'Top 3': 'Batman - The Telltale Series',
   'Top 4': 'SMITE®',
   'Top 5': 'FINAL FANTASY TYPE-0™ H

In [21]:
'''
    Exporto el dataset como un csv
'''
prueba1.to_csv('../data_set_limpio/modelo.csv',encoding='UTF-8',index=False)

In [22]:
'''
    Releo el archivo, para luego comprimirlo y exportarlo ahora en gzip.
'''
with open('../data_set_limpio/modelo.csv', 'rb') as f_in, gzip.open('../data_set_limpio/modelo.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)

In [425]:
'''
    Bloque de codigo para realizar el llamado al modelo de machine learning
'''
with open('../Modelo/modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(model, file)